## Online Simulation-Based Digital Twin: Nibelungenbrücke

This notebook introduces an online simulation-based digital twin developed for the Nibelungenbrücke bridge. The orchestration system is designed to interact dynamically with the user by gathering key inputs; such as time, physical parameters, and spatial positions to perform real-time simulations of the bridge structure.

The simulations are powered by the FenicSXConcrete package and support both thermomechanical and structural deflection analyses. This framework enables physics-based modeling and continuous digital monitoring of the bridge's behavior.




Please run the code below once to install the packages!

In [ ]:
#import sys
#!{sys.executable} -m pip install git+https://github.com/BAMresearch/FenicsXConcrete pint gmsh pytest jsonschema pandas pyproj tqdm pvlib

Following code snippet adjusts the current working directory and modifies the system path to ensure proper module resolution.

In [ ]:
import os
import sys

original_cwd = os.getcwd()
root_dir = os.getcwd()
orchestrator_dir = os.path.join(root_dir, 'nibelungenbruecke', 'scripts', 'digital_twin_orchestrator')
os.chdir(orchestrator_dir)
sys.path.insert(0, root_dir)

from nibelungenbruecke.scripts.digital_twin_orchestrator.orchestrator import Orchestrator

In [ ]:
import importlib
import nibelungenbruecke.scripts.digital_twin_orchestrator.orchestrator as orch

importlib.reload(orch)


### Definition of input parameters

This dictionary defines the configuration for running a digital twin simulation, including model selection, temporal settings, sensor definitions, and simulation options.

In [ ]:
simulation_parameters = {
    'simulation_name': 'TestSimulation',
    'model': 'TransientThermal_1',
    'start_time': '2023-08-11T08:00:00Z',
    'end_time': '2023-08-11T16:10:00Z',
    'time_step': '10min',
    'virtual_sensor_positions': [
        {'x': 0.0, 'y': 0.0, 'z': 0.0, 'name': 'Sensor1'},
        {'x': 1.0, 'y': 0.0, 'z': 0.0, 'name': 'Sensor2'},
        {'x': 1.78, 'y': 0.0, 'z': 26.91, 'name': 'Sensor3'},
        {'x': -1.83, 'y': 0.0, 'z': 0.0, 'name': 'Sensor4'}
    ],
    'plot_pv': True,
    'full_field_results': True, # Set to True if you want full field results, the simulation will take longer and the results will be larger.
    'uncertainty_quantification': False, # Set to True if you want uncertainty quantification, the simulation will take longer and the results will be larger.
}

The Orchestrator class acts as the central controller for managing the entire digital twin simulation workflow. It handles configuration, setup, execution, and post-processing of simulations based on the provided input parameters.<br>
<br>
Orchestrator initialization with respect to the given parameters:

In [ ]:
orchestrator = orch.Orchestrator(simulation_parameters)

Provide your key to MKP's API to registers it with the Orchestrator instance for secure communication with the backend services.

In [ ]:
key=input("\nEnter the code to connect API: ").strip()
#key = ""
orchestrator.set_api_key(key)

Before proceeding with the simulation, the `load()` method verifies whether the coordinates of the virtual sensors lie within the boundaries of the mesh domain.

In [ ]:
#orchestrator.load(simulation_parameters)

The simulation is executed using:

In [ ]:

results = orchestrator.run()

### Results
The result at the sensors defined in simulation_parameters are plotted below.

In [ ]:
orchestrator.plot_virtual_sensor_data()

Plot comparison between model response and real sensors when available:

In [ ]:
orchestrator.plot_real_sensor_vs_virtual_sensor()

## Additional results (only if run)

### Full-field response (3D)

The full-field simulation results can be reached with the following method.<br>

! Interactive visualization within the interface is not yet supported but will be introduced soon. Meanwhile files can be downloaded from the specified paths printed below.

In [ ]:
orchestrator.plot_full_field_response(simulation_parameters["full_field_results"])

### Thermal Model Uncertainty Quantification

Uncertainty quantification for the thermal model has been introduced. To enable this feature:  
- Set the parameter `uncertainty_quantification` to `True`  
- Ensure the parameter `plot_pv` is set to `False`  

This configuration activates the uncertainty analysis.

In [ ]:
simulation_parameters = {
    'simulation_name': 'TestSimulation',
    'model': 'TransientThermal_1',
    'start_time': '2023-08-11T08:00:00Z',
    'end_time': '2023-08-13T08:10:00Z',
    'time_step': '10min',
    'virtual_sensor_positions': [
        {'x': 0.0, 'y': 0.0, 'z': 0.0, 'name': 'Sensor1'},
        {'x': 1.0, 'y': 0.0, 'z': 0.0, 'name': 'Sensor2'},
        {'x': 1.78, 'y': 0.0, 'z': 26.91, 'name': 'Sensor3'},
        {'x': -1.83, 'y': 0.0, 'z': 0.0, 'name': 'Sensor4'}
    ],
    'plot_pv': False,
    'full_field_results': True,
    'uncertainty_quantification': True,
}

Verify that all given virtual sensors fall within the mesh domain and then run the model.

In [ ]:
#orchestrator.load(simulation_parameters)
orchestrator.run(simulation_parameters)

In [ ]:
orchestrator.plot_virtual_sensor_data()

In [ ]:
orchestrator.plot_real_sensor_vs_virtual_sensor()

In [ ]:
orchestrator.plot_full_field_response(simulation_parameters["full_field_results"])

In [ ]:
os.chdir(original_cwd)
print("Working directory restored to:", original_cwd)